<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-изучение-данных" data-toc-modified-id="Загрузка-и-изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Константные-модели" data-toc-modified-id="Константные-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Константные модели</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>LightGBM</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#!pip install nltk

In [1]:
import sys

In [3]:
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import nltk
import spacy
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Подготовка

### Загрузка и изучение данных

Загрузим данные с сервера, так они будут запускаться в любой среде:

In [4]:
path='https://code.s3.yandex.net/datasets/'
file_name='toxic_comments.csv'

df = pd.read_csv(path+file_name)

Изменим максимальную длину столбца и максимальное отображаемое количество строк чтобы почитать комментарии:

In [5]:
pd.set_option('display.max_colwidth', 255)
pd.options.display.max_rows=1000

Изучим загруженный датасет:

In [6]:
df.head(10)

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired no...",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie d...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I h...",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [9]:
df.duplicated().sum()

0

Посчитаем количество токсичных комментариев и их долю:

In [10]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [11]:
df['toxic'].unique()

array([0, 1])

In [12]:
df['toxic'].value_counts()[1] / df['toxic'].value_counts()[0]*100

11.31876717871444

Ознакомимся с токсичными комментариями:

In [13]:
df.query('toxic==1').head(10)

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,"Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAs...",1
16,"Bye! \n\nDon't look, come or think of comming back! Tosser.",1
42,"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...\n\n1 - If you are anti-se...",1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
44,I'm Sorry \n\nI'm sorry I screwed around with someones talk page. It was very bad to do. I know how having the templates on their talk page helps you assert your dominance over them. I know I should bow down to the almighty administrators. But the...,1
51,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK THAT YOU CANT PUT DOWN???/ GET FUCK UP GET FUCKED UP. I'M FUCKED UP RIGHT NOW!,1
55,Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell!,1
56,=Tony Sidaway is obviously a fistfuckee. He loves an arm up his ass.,1
58,"My Band Page's deletion. You thought I was gone. \n\nDeleting the comment I posted on your 'talk page' does not delete my feelings for your hasty decision to delete my page. You, sir, are still a cock-sucking douche fuck. Sit back for a second and thi...",1


* Датасет содержит 159571 строку, каждая из которых - комментарий пользователя. 
* Есть 2 признака - сам текст и его токсичность (да/нет).
* Пропусков и дубликатов в данных не обнаружено
* Типы данных корректные, но в целях экономии памяти можно токсичность преобразовать в булев тип, он содержит только 1 или 0.
* Доля токсичных комментариев 11.32%, это означет, что при обучении моделей предстоит перебороть существенный дисбаланс классов
* Комментарии с toxic=1 действительно токсичные

### Предобработка данных

Изменим тип ключевого признака на булев:

In [14]:
df['toxic'] = df['toxic'].astype('bool')

Токенизируем и лемматизируем англоязычный текст:

In [15]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

In [16]:
# Функция со Stack Overflow для токенизации+лемматизации текста
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
df['lemm_text'] = df.text.apply(lemmatize_text)

In [17]:
df.head()

,text,toxic,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired no...",False,"[Explanation, Why, the, edits, made, under, my, username, Hardcore, Metallica, Fan, were, reverted?, They, weren't, vandalisms,, just, closure, on, some, GAs, after, I, voted, at, New, York, Dolls, FAC., And, please, don't, remove, the, template, from..."
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",False,"[D'aww!, He, match, this, background, colour, I'm, seemingly, stuck, with., Thanks., (talk), 21:51,, January, 11,, 2016, (UTC)]"
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",False,"[Hey, man,, I'm, really, not, trying, to, edit, war., It's, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page., He, seems, to, care, more, about, the, formatting, ..."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie d...",False,"["", More, I, can't, make, any, real, suggestion, on, improvement, -, I, wondered, if, the, section, statistic, should, be, later, on,, or, a, subsection, of, """"types, of, accidents"""", -I, think, the, reference, may, need, tidying, so, that, they, are,..."
4,"You, sir, are my hero. Any chance you remember what page that's on?",False,"[You,, sir,, are, my, hero., Any, chance, you, remember, what, page, that's, on?]"


На самом деле довольно слабая лемматизация, например он изменил matches на match, но оставил много других слов не в исходной форме. Чтобы лемматизация в nltk работа адекватно, нужно задавать форму речи. Очевидно, что задать форму речи для такого объема данных практически невозможно и я не вижу вариантов как это можно быстро автоматизировать. Но, к счастью, кто-то уже это сделал поэтому импортируем библиотеку, которая автоматически определяет форму речи и находит правильную лемму для каждого слова, т.е. по сути делает адекватную лемматизацию:

In [18]:
nlp = spacy.load('en_core_web_sm')

Применим spacy ко всему объему данных:

In [19]:
%%time
df['strong_lemm'] = df['text'].apply(lambda x: " ".join([w.lemma_ for w in nlp(x)]))

CPU times: user 56min 7s, sys: 11.1 s, total: 56min 18s
Wall time: 57min 53s


In [20]:
df.head()

,text,toxic,lemm_text,strong_lemm
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired no...",False,"[Explanation, Why, the, edits, made, under, my, username, Hardcore, Metallica, Fan, were, reverted?, They, weren't, vandalisms,, just, closure, on, some, GAs, after, I, voted, at, New, York, Dolls, FAC., And, please, don't, remove, the, template, from...","explanation \n why the edit make under -PRON- username Hardcore Metallica Fan be revert ? -PRON- be not vandalism , just closure on some gas after -PRON- vote at New York Dolls FAC . and please do not remove the template from the talk page since -PRON..."
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",False,"[D'aww!, He, match, this, background, colour, I'm, seemingly, stuck, with., Thanks., (talk), 21:51,, January, 11,, 2016, (UTC)]","d'aww ! -PRON- match this background colour -PRON- be seemingly stick with . thank . ( talk ) 21:51 , January 11 , 2016 ( UTC )"
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",False,"[Hey, man,, I'm, really, not, trying, to, edit, war., It's, just, that, this, guy, is, constantly, removing, relevant, information, and, talking, to, me, through, edits, instead, of, my, talk, page., He, seems, to, care, more, about, the, formatting, ...","hey man , -PRON- be really not try to edit war . -PRON- be just that this guy be constantly remove relevant information and talk to -PRON- through edit instead of -PRON- talk page . -PRON- seem to care more about the formatting than the actual info ."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie d...",False,"["", More, I, can't, make, any, real, suggestion, on, improvement, -, I, wondered, if, the, section, statistic, should, be, later, on,, or, a, subsection, of, """"types, of, accidents"""", -I, think, the, reference, may, need, tidying, so, that, they, are,...",""" \n More \n -PRON- can not make any real suggestion on improvement - -PRON- wonder if the section statistic should be later on , or a subsection of "" "" type of accident "" "" -I think the reference may need tidying so that -PRON- be all in the exact ..."
4,"You, sir, are my hero. Any chance you remember what page that's on?",False,"[You,, sir,, are, my, hero., Any, chance, you, remember, what, page, that's, on?]","-PRON- , sir , be -PRON- hero . any chance -PRON- remember what page that be on ?"


Так намного лучше, хотя и есть вопросы к "I ca n't", надеюсь так и должно быть. Но spacy работает с исходным текстом, а nltk с токенизированным, поэтому, на всякий случай, я бы прогнал ещё результат через функцию токенизации+лемматизации:

In [21]:
%%time
df['lemm_text'] = df.strong_lemm.apply(lemmatize_text)

CPU times: user 1min 15s, sys: 1.34 s, total: 1min 17s
Wall time: 1min 18s


In [22]:
df.head()

,text,toxic,lemm_text,strong_lemm
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired no...",False,"[explanation, why, the, edit, make, under, -PRON-, username, Hardcore, Metallica, Fan, be, revert, ?, -PRON-, be, not, vandalism, ,, just, closure, on, some, gas, after, -PRON-, vote, at, New, York, Dolls, FAC, ., and, please, do, not, remove, the, te...","explanation \n why the edit make under -PRON- username Hardcore Metallica Fan be revert ? -PRON- be not vandalism , just closure on some gas after -PRON- vote at New York Dolls FAC . and please do not remove the template from the talk page since -PRON..."
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",False,"[d'aww, !, -PRON-, match, this, background, colour, -PRON-, be, seemingly, stick, with, ., thank, ., (, talk, ), 21:51, ,, January, 11, ,, 2016, (, UTC, )]","d'aww ! -PRON- match this background colour -PRON- be seemingly stick with . thank . ( talk ) 21:51 , January 11 , 2016 ( UTC )"
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",False,"[hey, man, ,, -PRON-, be, really, not, try, to, edit, war, ., -PRON-, be, just, that, this, guy, be, constantly, remove, relevant, information, and, talk, to, -PRON-, through, edit, instead, of, -PRON-, talk, page, ., -PRON-, seem, to, care, more, abo...","hey man , -PRON- be really not try to edit war . -PRON- be just that this guy be constantly remove relevant information and talk to -PRON- through edit instead of -PRON- talk page . -PRON- seem to care more about the formatting than the actual info ."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie d...",False,"["", More, -PRON-, can, not, make, any, real, suggestion, on, improvement, -, -PRON-, wonder, if, the, section, statistic, should, be, later, on, ,, or, a, subsection, of, "", "", type, of, accident, "", "", -I, think, the, reference, may, need, tidying, s...",""" \n More \n -PRON- can not make any real suggestion on improvement - -PRON- wonder if the section statistic should be later on , or a subsection of "" "" type of accident "" "" -I think the reference may need tidying so that -PRON- be all in the exact ..."
4,"You, sir, are my hero. Any chance you remember what page that's on?",False,"[-PRON-, ,, sir, ,, be, -PRON-, hero, ., any, chance, -PRON-, remember, what, page, that, be, on, ?]","-PRON- , sir , be -PRON- hero . any chance -PRON- remember what page that be on ?"


Теперь в lemm_text все комментарии лемматизированны адекватно, strong_lemm и исходный текст можно удалить.

In [23]:
df['strong_lemm'] = df['text']

In [24]:
df = df.drop(['strong_lemm', 'text'], axis=1)

In [25]:
data = df.copy()

In [26]:
data = data[['lemm_text', 'toxic']]

In [27]:
data.head()

,lemm_text,toxic
0,"[explanation, why, the, edit, make, under, -PRON-, username, Hardcore, Metallica, Fan, be, revert, ?, -PRON-, be, not, vandalism, ,, just, closure, on, some, gas, after, -PRON-, vote, at, New, York, Dolls, FAC, ., and, please, do, not, remove, the, te...",False
1,"[d'aww, !, -PRON-, match, this, background, colour, -PRON-, be, seemingly, stick, with, ., thank, ., (, talk, ), 21:51, ,, January, 11, ,, 2016, (, UTC, )]",False
2,"[hey, man, ,, -PRON-, be, really, not, try, to, edit, war, ., -PRON-, be, just, that, this, guy, be, constantly, remove, relevant, information, and, talk, to, -PRON-, through, edit, instead, of, -PRON-, talk, page, ., -PRON-, seem, to, care, more, abo...",False
3,"["", More, -PRON-, can, not, make, any, real, suggestion, on, improvement, -, -PRON-, wonder, if, the, section, statistic, should, be, later, on, ,, or, a, subsection, of, "", "", type, of, accident, "", "", -I, think, the, reference, may, need, tidying, s...",False
4,"[-PRON-, ,, sir, ,, be, -PRON-, hero, ., any, chance, -PRON-, remember, what, page, that, be, on, ?]",False


Далее очистим текст от лишних знаков и чисел:

In [28]:
def remove_punctuation_1(text):
    no_punct = " ".join([c for c in text if c not in string.punctuation])
    return no_punct
def remove_punctuation_2(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [29]:
%%time
data['lemm_text'] = data['lemm_text'].apply(lambda x: remove_punctuation_1(x)) # очистка всех знаков

CPU times: user 2.82 s, sys: 4 ms, total: 2.82 s
Wall time: 2.84 s


In [30]:
data.lemm_text = data.lemm_text.str.replace('\d+', '') # удаление всех чисел из текста

In [31]:
%%time
# очистка вновь появившихся знаков после удаления чисел (разделители)
data['lemm_text'] = data['lemm_text'].apply(lambda x: remove_punctuation_2(x)) 

CPU times: user 10.1 s, sys: 40 ms, total: 10.2 s
Wall time: 10.3 s


Преобразуем все слова в датафрейме к нижнему регистру потому что этого требует TfidfVectorizer по умолчанию:

In [32]:
data['lemm_text'] = data['lemm_text'].apply(lambda x: x.lower())

In [33]:
data.head()

,lemm_text,toxic
0,explanation why the edit make under pron username hardcore metallica fan be revert pron be not vandalism just closure on some gas after pron vote at new york dolls fac and please do not remove the template from the talk page since pron be retire now,False
1,daww pron match this background colour pron be seemingly stick with thank talk january utc,False
2,hey man pron be really not try to edit war pron be just that this guy be constantly remove relevant information and talk to pron through edit instead of pron talk page pron seem to care more about the formatting than the actual info,False
3,more pron can not make any real suggestion on improvement pron wonder if the section statistic should be later on or a subsection of type of accident i think the reference may need tidying so that pron be all in the exact same format ie date format et...,False
4,pron sir be pron hero any chance pron remember what page that be on,False


Теперь остались только правильно лемматизированные слова, которые отделены пробелами. Разделим выборку на обучающую и тестовую: 

In [34]:
random_state = 42

In [35]:
train, test = train_test_split(data, test_size=0.25, random_state=random_state, stratify=data['toxic'])

In [36]:
print("В обучающую выборку попало данных, %: ", len(train)/len(data)*100)
print("В тестовую выборку попало данных, %: ", len(test)/len(data)*100)
print("Объектов целевого признака в обучающей выборке, %:", \
      (train['toxic'].value_counts()[1] / train['toxic'].value_counts()[0]*100))
print("Объектов целевого признака в тестовой выборке, %:", \
      (test['toxic'].value_counts()[1] / test['toxic'].value_counts()[0]*100))

В обучающую выборку попало данных, %:  74.99984332992837
В тестовую выборку попало данных, %:  25.000156670071625
Объектов целевого признака в обучающей выборке, %: 11.319052358407204
Объектов целевого признака в тестовой выборке, %: 11.317911655551525


Добавим список стоп-слов и сохраним в переменной, который далее передадим в векторизатор текста:

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
stop_words = set(stopwords.words('english'))

## Обучение

Воспользуемся векторизатором TfidfVectorizer, в который передадим стоп-слова:

In [39]:
vectorizer = TfidfVectorizer(stop_words=stop_words)

<i> Оставил дефолтный ngrams несмотря на то что с ngrams=(1,2) результат чуть лучше, так как очень сильно раздувается матрица и значительно увеличивается время обучения

Результат обучения и трансформации списка текстов сохраним как признаки для обучения:

In [40]:
features_train = vectorizer.fit_transform(train['lemm_text'].values) 

В target_train сохраним целевой признак обучающей выборки:

In [41]:
target_train = train['toxic']

Трансформируем обученным векторайзером тексты тестовой выборки и сохраним в переменной features_test:

In [42]:
features_test = vectorizer.transform(test['lemm_text'].values)

В target_test по аналогии сохраним целевой признак тестовой выборки:

In [43]:
target_test = test['toxic']

Перейдем к обучению самих моделей

### Логистическая регрессия

Обучим логистическую регрессию на трейне. Для этой модели не будем использовать подбор гиперпараметров по сетке, однако установим по умолчанию solver и балансировку классов:

In [44]:
model_logistic_regression = LogisticRegression(random_state=random_state, solver='sag', class_weight='balanced') 
model_logistic_regression.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

<i> sag - единственный solver при котором логистическая регрессия не выдаёт ошибки/предупреждения из-за размерности матрицы

Сделаем предсказания на тестовой выборке:

In [45]:
predictions_logistic_regression = model_logistic_regression.predict(features_test)

Посчитаем f1 на тесте.

In [46]:
f1_score(target_test, predictions_logistic_regression)

0.7420419461300969

В исходном датасете 11.3% положительных значений целевого признака (т.е. единиц). Посмотрим сколько напредиктила модель:

In [47]:
pd.Series(predictions_logistic_regression).value_counts()[1] / pd.Series(predictions_logistic_regression).value_counts()[0]

0.15444495890728094

Показатель завышен. Это означает, что вероятнее всего в балансе precision/recall будет перекос в пользу recall - модель реже ошибается тогда, когда истинный ответ "1", а модель определяет его как "0", то есть не определяет токсичный комментарий тогда, когда он действительно таковым является. На мой взгляд, такая ситуация конкретно в этой бизнес задаче - меньшее из зол, ведь мы отправляем комментарий на модерацию и если модератор решит, что комментарий не токсичный, он просто оставит его опубликованным. Обратная сторона - больше работы у модератора. Как бы то ни было, такая метрика очень близка, но всё ещё не соответствует условию по ТЗ (f1>0.75). Также отмечу, что без балансировки классов доля единиц будет наоборот занижена (~8%) и f1 хуже.

Чтобы убедиться в выводах с хорошим показателем данной модели по recall и слабым по precision дополнительно посчитаем и эти метрики:

In [48]:
precision_score(target_test, predictions_logistic_regression)

0.6529885703578789

In [49]:
recall_score(target_test, predictions_logistic_regression)

0.8592209072978304

Выводы верны, причём перекос в пользу recall довольно сильный. Для удобства в дальнейшем напишем функцию для нахождения метрик:

In [50]:
def model_test_score(target_test, predictions):
    return display('Лучшая F1 мера на тестовой выборке:', f1_score(target_test, predictions)),\
           display ('Precision:', precision_score(target_test, predictions)),\
           display ('Recall:', recall_score(target_test, predictions))

### Константные модели

В принципе, в данном проекте не вижу смысла проверять константные модели потому что:
* Если модель будет проставлять метки сильного класса (нули), то это будет равносильно её отсутствию
* Если модель будет проставлять метки слабого класса (единицы), то это всё равно что модератор проверял бы абсолютно все комментарии, то есть это тоже равносильно её отсутствию
* Если модель будет проставлять случайно 1 или 0, то очевидно, что у неё будет плохая метрика как минимум из-за дисбаланса классов
* Есть четкое требование по ТЗ (f1>0.75)

Но на всякий случай всё таки проверим константные модели и оставим в тетрадке лучшую из них:

In [51]:
model_dummy = DummyClassifier(strategy="uniform")
model_dummy.fit(features_train, target_train)
predictions_dummy = model_dummy.predict(features_test)

In [52]:
model_test_score(target_test, predictions_dummy);

'Лучшая F1 мера на тестовой выборке:'

0.1691795085381091

'Precision:'

0.10178410343790718

'Recall:'

0.5007396449704142

Как и ожидалось, рандомная дамми-модель самая лучшая из константных моделей, но F1 у неё при этом очень низкая.

### Решающее дерево

Подберем параметры для дерева с использованием RandomSearchCV:

In [53]:
params_tree = {
            'max_depth':[2, 4, 6, 8, 10, 14, 20],
            'max_features': ['auto', 'sqrt', 'log2', None],
            'min_samples_leaf': [1, 3, 5, 7, 9, 11, 15],
            'min_samples_split': [2, 4, 6, 8, 10, 14, 20],
            'class_weight': ['balanced', None],
              }

Интересно, что балансировка классов в данном случае не нужна. Обучим модель решающего дерева с найденными параметрами и сделаем предикты:

In [54]:
model_tree = DecisionTreeClassifier(max_depth=20, min_samples_leaf=3, min_samples_split=14, random_state=42)
model_tree.fit(features_train, target_train)
predictions_tree = model_tree.predict(features_test)

Посчитаем метрики <b> дерева</b> на тесте:

In [55]:
model_test_score(target_test, predictions_tree);

'Лучшая F1 мера на тестовой выборке:'

0.6455755677368834

'Precision:'

0.8849291541434092

'Recall:'

0.5081360946745562

* Дерево показазывает заметно хуже F1, чем логистическая регрессия. При этом наблюдается противоположная ситуация с балансом Precision/Recall - сильный перекос в сторону Precision.

### Случайный лес

Подберем параметры для случайного леса с использованием RandomSearchCV:

In [56]:
params_forest = {
            'n_estimators': [100, 150, 200, 250],
            'max_depth':[6, 8, 10, 14, 20, 24],
            'max_features': ['auto', 'sqrt', 'log2', None],
            'min_samples_leaf': [1, 3, 5, 7, 9, 11, 15],
            'min_samples_split': [2, 4, 6, 8, 10, 14, 20],
            'class_weight': ['balanced', None],
              }

Обучим модель случайного леса с найденными параметрами и сделаем предикты:

In [57]:
model_forest = RandomForestClassifier(max_depth=24, max_features=None, min_samples_leaf=5,
                       min_samples_split=4, n_estimators=200, random_state=42, n_jobs=-1)
model_forest.fit(features_train, target_train)
predictions_forest = model_forest.predict(features_test)

Найдем метрики <b>случайного леса</b> на тестовой выборке:

In [58]:
model_test_score(target_test, predictions_forest);

'Лучшая F1 мера на тестовой выборке:'

0.6714372716199757

'Precision:'

0.8777866242038217

'Recall:'

0.5436390532544378

Результат работы леса как обычно лучше дерева (0.64), но всё ещё уступает логистической регрессии (0.74). Также ожидаемо наблюдается перекос в Precision.

### LightGBM

Обучим модель градиентного бустинга. Начнем с LightGBM из-за невероятной скорости его работы. Если удастся достичь нужного результата, то другие модели бустинга проверять не будем. В этом проекте на вход подаётся очень большая матрица и обучение (особенно подбор гиперпараметров) занимает очень много времени.

In [59]:
params_lgb = {
    'num_leaves' : [5, 7, 10, 15, 25],
    'learning_rate' : [0.03, 0.1, 0.2, 0.3],
    'max_depth' : [4, 8, 12, 16, 20, 24, 30],
    'n_estimators': [150, 200, 250, 300, 350],
}

F1 на кросс-валидации лучше, чем у других моделей, обучим модель с подобранными гиперпараметрами и проверим на тесте:

In [61]:
model_lgb = lgb.LGBMClassifier(random_state=random_state, n_jobs=-1,
                              num_leaves=25,
                              n_estimators=350,
                              max_depth=20,
                              learning_rate=0.2)

model_lgb.fit(features_train, target_train)

predictions_lgb = model_lgb.predict(features_test)

In [62]:
model_test_score(target_test, predictions_lgb);

'Лучшая F1 мера на тестовой выборке:'

0.7766990291262136

'Precision:'

0.8719680687749463

'Recall:'

0.7001972386587771

F1 почти 0.78! Но, к сожелнию, в балансе precision/recall перекос в пользу Precision. Это означает, что модель будет реже реагировать на токсичные комментарии, чем хотелось бы. Зато у модератора будет меньше работы. Тем не менее, попробуем откалибровать модель. Самый простой способ сделать это автоматически - установить class_weight=balanced. Тогда модель будет более уверена в ответах слабого класса и нарисует их больше. Не очень хочется заново перебирать гиперпараметры по сетке (хотя по хорошему именно так нужно поступить), поэтому попробуем просто добавить текущей модели уверенности через class_weight:

In [63]:
model_lgb_balanced = lgb.LGBMClassifier(random_state=random_state, n_jobs=-1,
                              num_leaves=25,
                              n_estimators=350,
                              max_depth=20,
                              learning_rate=0.2,
                              class_weight='balanced')

model_lgb_balanced.fit(features_train, target_train)

predictions_lgb_balanced = model_lgb_balanced.predict(features_test)

In [64]:
model_test_score(target_test, predictions_lgb_balanced);

'Лучшая F1 мера на тестовой выборке:'

0.7580735277684657

'Precision:'

0.6932352340077662

'Recall:'

0.8362919132149902

Порог F1 в 0.75 пройден и одновременно удалось достичь ожидаемого результата в балансе классов.

## Выводы

В проекте обработан большой массив текстовых данных с разметкой: комментарии пользователей интернет-магазина "Викишоп". На его основе обучено несколько моделей, которые помогут находить токсичные комментарии пользоваталей. Метрики моделей сведем в таблицу:

<a href="https://imgur.com/hf2VSid"><img src="https://i.imgur.com/hf2VSid.png" title="source: imgur.com" /></a>

Требуемый уровень качества по ТЗ (f1>0.75) превосходят две модели - LightGBM и LightGBM balanced. Я бы рекомендовал заказчику внедрить вторую модель LightGBM balanced, так как у неё намного выше метрика Recall. Это означает, что по сравнению с первой, эта модель реже будет ошибаться в определении действительно токсичного комментария как нетоксичного, но чаще будет отправлять на модерацию нетоксичный комментарий, ошибочно решив, что он токсичный. Таким образом, с точки зрения улучшения качества контента на сайте (минимизации количества токсичных комментариев) лучше подходит вторая модель. Однако, если стоит задача снизить нагрузку на модераторов, то лучше использовать первую - она будет отправлять на модерацию намного меньше комментарив из-за более высокой точности (Precision), но при этом чаще будет игнорировать токсичные комментарии (примерно в 30% случаев).

Ещё короче и понятнее будет в среднем так:

<br>Модель 1 (LightGBM):
* Отправляет 8.8% комментариев на модерацию (меньше нагрузка на модераторов), при этом лишь 12.1% из них не являются токсичными
* Не замечает ~30% действительно токсичных комментариев на сайте

<br>Модель 2 (LightGBM balanced):
* Отправляет 13.6% комментариев на модерацию (больше нагрузка на модераторов), при этом 29.7% из них не являются токсичными
* Не замечает лишь ~17% действительно токсичных комментариев на сайте

<br><b>Модель 1 - про оптимизацию затрат на модерацию
<br> Модель 2 - про качество контента на сайте